### Code Explaination For Demo

## ⚙️ How the Code Works

### Reader Process (reader.py)

```python
# Continuously reads the file in a loop
while True:
    state = read_state()      # Read from shared_state.json
    display_state(state)      # Show the data
    sleep(10)                # Wait before reading again
```

### Writer Process (writer.py)

```python
# Updates are made in 3 steps
state = read_state()          # 1. Read current state
state["message"] = "Hello!"   # 2. Modify the data
write_state(state)            # 3. Write back to file
```

In [ ]:
"""reader.py
# File Based Interprocess Communication (IPC)
**A lightweight IPC method suitable for simple CLI applications where sub-second latency is acceptable. It uses file-based communication.**
"""

# ==============================================
# FILE BASED IPC DEMONSTRATION: READER PROCESS
# ==============================================

import json # for reading JSON data
from pathlib import Path# for file path handling
from time import sleep# for adding delays between reads

from rich.text import Text
from rich.console import Console

console = Console()

#==========================================================================================================================
#TODO: Create a Constant for the shared state file path, must match the writer's path that points to shared state file
#==========================================================================================================================
    # Explanation: SHARED_STATE_FILE = Path(__file__).parent / "ipc_state.json"
        # this is needed in both the reader and writer processes because they both need to access the same file
        # Path(), provides an object-oriented approach to handling file system paths
            # it will create a Path object representing the directory of the current script
        # Path(__file__), will give the full path to the current script `reader.py`:
            # ....\Python-File-Based-IPC\Demo\reader.py
            # __file__ is a special variable that holds the path of the current script
        # Path(__file__).parent, gets the directory part of the path which is, `Demo/` in this case
            # ensure it is relative to the current script's location
            # ...\Python-File-Based-IPC\Demo
        # Path(__file__).parent / "ipc_state.json", will append "ipc_state.json" to the directory path
            # ...\Python-File-Based-IPC\Demo\ipc_state.json
            # and "ipc_state.json" is the filename used for storing the shared state.
# print(f"Shared state file path: {SHARED_STATE_FILE}")
#==========================================================================================================================
SHARED_STATE_FILE = Path(__file__).parent / "ipc_state.json"

# ipc_state.json: you do not have to create the file beforehand, the writer process will create it when it writes the first time
# but you can create an empty file manually if you want to avoid the FileNotFoundError on the first read attempt


#==========================================================================================================================
# TODO: Implement a function to initialize the reader process
# It can include any setup logic needed before reading the shared state
#==========================================================================================================================
def initialize_reader():
    #=========================================================================================================== 
    # Check if the shared state file exists, if not create it with initial content
    # Initial content can be an empty dictionary or some default values
    # Here we create it with some default values
    # then write the initial state to the file and print a confirmation message
    #   json.dump() will serialize the dictionary to JSON format and write it to the file
    #   serialize: convert a data structure into a format that can be easily stored or transmitted
    #===========================================================================================================
    initialized_state_message = Text("Initialized shared_state.json", style="bold green")

    if not SHARED_STATE_FILE.exists():
        initial_state = {
            "reader_message": "Reader process initialized successfully.",
            "writer_message": "This will be updated by the writer process. Waiting for writer..."
        }
        with open(SHARED_STATE_FILE, 'w') as f:
            json.dump(initial_state, f, indent=2)
        console.print(initialized_state_message)


#==========================================================================================================================
# TODO: Implement a function to read the shared state from the file and handle the case where the file does not exist yet
#   the function should:
#   try to open and read the JSON file
#   parse the JSON data into a Python dictionary
#   return the dictionary
#   if the file does not exist, it should:
#   Handle the FileNotFoundError exception
#   print a message indicating that the shared state file was not found
#   because the writer process may not have created it yet
#==========================================================================================================================
def read_state():
    #==========================================================================================================================
    # `with`:
    #  is a context manager that ensures the file is properly closed after its suite finishes
    # the context manager handles opening and closing the file automatically
    # a context manager is a construct that allows for setup and teardown actions around a block of code
    # it is commonly used for resource management, such as file handling, network connections, and database connections
    # in this case, it ensures that the file is closed after reading, even if an error occurs during the read operation
    #==========================================================================================================================

    #==========================================================================================================================
    # `json.load(file)`:
    #  reads JSON data from a file object and parses it into a Python dictionary
    # it takes a file object as an argument and reads the entire content of the file
    # then it converts the JSON formatted string into a corresponding Python data structure
    #==========================================================================================================================
    try:
        with open(SHARED_STATE_FILE, "r") as file:
            return json.load(file)
    except FileNotFoundError:
        print("Shared state file not found.")
        return None
    

#==========================================================================================================================
# TODO: Implement a function to display the shared state
#==========================================================================================================================
def display_state(state):
    # state will be a dictionary
    console.print("[dim yellow]Current Shared State:[/dim yellow]")
    for key, value in state.items():
        key_styled = Text(key, style="bold cyan")
        value_styled = Text(value, style="magenta")
        # Don't use f-string which converts Text to plain string
        # Use console.print with Text objects directly
        console.print("  ", key_styled, ": ", value_styled)



def main():
    initialize_reader()# ensure the shared state file is initialized

    try:
        while True:
            # Read the current state from the file
            state = read_state()
            
            if state:
                # Display the state
                display_state(state)
            
            # Wait before reading again
            # This prevents excessive file I/O
            sleep(5)

    except KeyboardInterrupt:
        console.print("[bold red]Reader stopped[/bold red]")

if __name__ == "__main__":
    main()

In [ ]:
"""writer.py
# File Based Interprocess Communication (IPC)
**A lightweight IPC method suitable for simple CLI applications where sub-second latency is acceptable. It uses
"""


import json # for reading JSON data
from pathlib import Path# for file path handling

from rich.text import Text
from rich.prompt import Prompt
from rich.console import Console

console = Console()

# Path to the shared state file - must match the reader's path
SHARED_STATE_FILE = Path(__file__).parent / "ipc_state.json"


#==========================================================================================================================
# TODO: Implement a function to read the shared state from the file
#==========================================================================================================================
def read_shared_state():
    try:
        with open(SHARED_STATE_FILE, "r") as file:
            return json.load(file)
    except FileNotFoundError:
        print("Shared state file not found.")
        return {
            "message": "Hello from writer process!",
        }
    
def update(new_message):
    # Read the current state
    state = read_shared_state()
    
    # Update the message
    state["writer_message"] = new_message
    
    # Write the updated state back to the file
    with open(SHARED_STATE_FILE, "w") as file:
        json.dump(state, file, indent=2)
    
    message_styled = Text("Updated message to:", style="yellow")
    message_value_styled = Text(new_message, style="bold cyan")
    console.print(message_styled, message_value_styled)

def main():
    while True:
        try:
            # Get user input
            user_input = Prompt.ask("[bold magenta]Enter your message[/bold magenta]").strip()
            
            if not user_input:
                continue
            else:
                # Treat as a message
                update(user_input)

        except KeyboardInterrupt:
            console.print("\n[bold red]Writer stopped[/bold red]")
            break

if __name__ == "__main__":
    main()